In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset

In [2]:
def calc_prob(channels, ydim, xdim, freqID, src_ID, noise, noise_level):
    
    ref = 'BON'
    
    #Pre-allocate array with probabilities
    probs_tw = np.zeros((xdim*ydim))
    
    # Energy ratios are calculated for each component in respect to the chosen
    # reference station
    for ch in channels:
        
        # Energy ratios from simulated signal
        if ch == 'Z':
            direc_comp = 'data/simu/vertZ/'
        elif ch == 'E':
            direc_comp = 'data/simu/horzE/'
        elif ch == 'N':
            direc_comp = 'data/simu/horzN/'
            
            
        if ch == 'Z':
            stas_name = ['BOR', 'DSO', 'SNE']
        else:
            stas_name = ['BOR', 'SNE']

        nRatios = len(stas_name)
        ratio_obs = np.zeros(nRatios)   
        
        path_simu_ref = direc_comp + ref + '/' 
        energy_simu_ref = np.loadtxt(path_simu_ref + freqID + '/energy_Fz.txt')
        #observed energy
        energy_curr_ref  = energy_simu_ref[src_ID]
        if noise:
            energy_curr_ref += np.abs(np.random.normal(0,noise_level))
        #simulated energy catalogue
        ###energy_simu_ref = np.reshape(energy_simu_ref,(ydim,xdim))
        ratios_simu = np.zeros((ydim*xdim,nRatios)) 
        
        for idxSrc, sta in enumerate(stas_name):
            path_simu = direc_comp  + sta + '/' 
            energy_simu = np.loadtxt(path_simu + freqID + '/energy_Fz.txt')
            #observed energy
            energy_curr = energy_simu[src_ID] 
            if noise:
                print('HI')
                energy_curr += np.abs(np.random.normal(0,noise_level))
            ratio_obs[idxSrc] = energy_curr/energy_curr_ref
            #simulated energy catalogue
            ###energy_simu = np.reshape(energy_simu,(ydim,xdim))
            ratios_simu[:,idxSrc] = energy_simu/energy_simu_ref
                    
        # Error between observed and simulated energy ratios
        for ii in range(xdim*ydim):
            for kk in range(nRatios):
                probs_tw[ii] += 1./nRatios * (
                    np.abs(np.log10(ratios_simu[ii,kk]/ratio_obs[kk])))

    # Probability is defined as inverse of error, normalized by 
    # number of channels
    probs_tw = 1./(probs_tw/len(channels))

    return(probs_tw)

In [3]:
def exponential(x, a, k, b):
    return a*np.exp(x*k) + b

In [4]:
src_IDs = [3398, 5606, 9959, 9896, 4069]
src_ID  = src_IDs[3]
src_ID = 3000
#src_ID = 5622

channels = [['E', 'N', 'Z'], ['Z']]
freqID   = 'BP3to7'

decay_level = 0.75

noise = False

# noise level
if freqID == 'BP3to7':
    noise_level = 1.025e-24
elif freqID == 'BP8to12':
    noise_level = 1.050e-24
elif freqID == 'BP13to17':
    noise_level = 3.277e-26
    

xdim = 121
ydim = 101

## Load topography
topo   = np.loadtxt('data/DEM_PF_10m_cut.dat',skiprows=6)
dx = 10
dy = 10
xx = np.arange(0.,2100.+dx,dx)
yy = np.arange(0.,1800.+dy,dy)
levels = np.arange(2190., 2650, 20)
### Load stations
srcs   = np.loadtxt('data/stations.txt')
srcs_array = ['BON', 'BOR', 'DSO', 'SNE']
xmin =  640.
xmax = 1840.
ymin =  400.
ymax = 1400.

In [5]:
# Loading grid positions
data = pd.read_csv('data/positions.txt', sep='\s+', 
                   header=None, names=['x', 'y', 'z']) 
pickX    = data.loc[src_ID,'x']    
pickY    = data.loc[src_ID,'y']
data['radius'] = np.sqrt((data.x-data.loc[src_ID,'x'])**2 + (data.y-data.loc[src_ID,'y'])**2)

In [6]:
for c, ichannels in enumerate(channels):
    
    probs = calc_prob(ichannels, ydim, xdim, freqID, src_ID, noise, noise_level)
    data['probs'+str(c)]  = probs
    
    #radius = np.array(data.radius)
    radius = np.delete(np.array(data.radius), src_ID)
    probs  = np.delete(probs,  src_ID)
    
    popt_exponential, pcov_exponential = optimize.curve_fit(exponential, radius, probs, p0=[10, -0.5, 0])
    perr_exponential = np.sqrt(np.diag(pcov_exponential))
    
    data['res_a'+str(c)]     = popt_exponential[0]
    data['res_a_err'+str(c)] = perr_exponential[0]
    data['res_k'+str(c)]     = popt_exponential[1]
    data['res_k_err'+str(c)] = perr_exponential[1]
    data['res_b'+str(c)]     = popt_exponential[2]
    data['res_b_err'+str(c)] = perr_exponential[2]
    
    data['xhalf'+str(c)] = np.log(decay_level)/popt_exponential[1]
    data['xhalf_err'+str(c)] = np.abs( np.log(decay_level)/(popt_exponential[1]**2) * perr_exponential[1])


/home/julian/Programs/anaconda2/envs/obspy/lib/python3.7/site-packages/ipykernel_launcher.py:60: RuntimeWarning: divide by zero encountered in true_divide
/home/julian/Programs/anaconda2/envs/obspy/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/home/julian/Programs/anaconda2/envs/obspy/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in multiply
  


In [7]:
data

,x,y,z,radius,probs0,res_a0,res_a_err0,res_k0,res_k_err0,res_b0,...,xhalf_err0,probs1,res_a1,res_a_err1,res_k1,res_k_err1,res_b1,res_b_err1,xhalf1,xhalf_err1
0,640.0,400.0,671.8667,989.545350,0.844996,15.742742,0.4285,-0.031426,0.000691,1.539572,...,0.20138,0.940222,12.365459,0.507755,-0.022892,0.000781,1.58207,0.014636,12.566846,0.428739
1,650.0,400.0,672.2667,979.846927,0.857507,15.742742,0.4285,-0.031426,0.000691,1.539572,...,0.20138,0.956488,12.365459,0.507755,-0.022892,0.000781,1.58207,0.014636,12.566846,0.428739
2,660.0,400.0,672.6667,970.154627,0.866437,15.742742,0.4285,-0.031426,0.000691,1.539572,...,0.20138,0.962998,12.365459,0.507755,-0.022892,0.000781,1.58207,0.014636,12.566846,0.428739
3,670.0,400.0,673.0667,960.468636,0.866978,15.742742,0.4285,-0.031426,0.000691,1.539572,...,0.20138,0.958703,12.365459,0.507755,-0.022892,0.000781,1.58207,0.014636,12.566846,0.428739
4,680.0,400.0,673.8667,950.789146,0.862727,15.742742,0.4285,-0.031426,0.000691,1.539572,...,0.20138,0.957848,12.365459,0.507755,-0.022892,0.000781,1.58207,0.014636,12.566846,0.428739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12216,1800.0,1400.0,611.2667,785.875308,0.818231,15.742742,0.4285,-0.031426,0.000691,1.539572,...,0.20138,0.723039,12.365459,0.507755,-0.022892,0.000781,1.58207,0.014636,12.566846,0.428739
12217,1810.0,1400.0,606.8667,788.479550,0.832775,15.742742,0.4285,-0.031426,0.000691,1.539572,...,0.20138,0.723796,12.365459,0.507755,-0.022892,0.000781,1.58207,0.014636,12.566846,0.428739
12218,1820.0,1400.0,603.2667,791.201618,0.840642,15.742742,0.4285,-0.031426,0.000691,1.539572,...,0.20138,0.728130,12.365459,0.507755,-0.022892,0.000781,1.58207,0.014636,12.566846,0.428739
12219,1830.0,1400.0,598.8667,794.040301,0.843751,15.742742,0.4285,-0.031426,0.000691,1.539572,...,0.20138,0.737001,12.365459,0.507755,-0.022892,0.000781,1.58207,0.014636,12.566846,0.428739


In [8]:
%matplotlib widget

decay_level = 0.75

fig, axes = plt.subplots(1,len(channels), figsize=(10,5), sharey=True)

for c, (ichannels, ax) in enumerate(zip(channels,axes)):
        
    print(data['xhalf'+str(c)][0])
    print(data['xhalf_err'+str(c)][0])

    
    ax.scatter(data.radius, data['probs'+str(c)]-data['res_b'+str(c)], c='C'+str(c))
    
    r = np.arange(0., data.radius.max(), 1.)
    #p = exponential(r, data['res_a'+str(c)][0], data['res_k'+str(c)][0], data['res_b'+str(c)][0])
    #ax.plot(r, p-data['res_b'+str(c)][0], 'r')
    p = exponential(r, data['res_a'+str(c)][0], data['res_k'+str(c)][0], 0.)
    ax.plot(r, p, 'r')

    
    y0 = data['res_a'+str(c)][0]
    ax.vlines(data['xhalf'+str(c)][0], ymin=0., ymax=y0*decay_level, color='k', ls='--')
    ax.hlines(y0*decay_level, xmin=0., xmax=data['xhalf'+str(c)][0], color='k', ls='--')
    
    ax.set_title(ichannels)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

9.154141303651135
0.2013797267019241
12.566845645214991
0.4287389336237341


In [11]:
data = data.drop([src_ID])
np.where(data.isin([np.nan, np.inf, -np.inf]).values)

(array([], dtype=int64), array([], dtype=int64))

In [12]:
popt_exponential, pcov_exponential = optimize.curve_fit(exponential, data.radius, data.probs, p0=[10, -0.5, 0])
perr_exponential = np.sqrt(np.diag(pcov_exponential))
print('pre-exponential factor = {:2.2} (+/-) {:2.2}'.format(popt_exponential[0], perr_exponential[0]))
print('rate constant = {:2.2} (+/-) {:2.2}'.format(popt_exponential[1], perr_exponential[1]))

#y0 = exponential(0., popt_exponential[0], popt_exponential[1], popt_exponential[2])
#xhalf = np.log((y0/2 - popt_exponential[2])/popt_exponential[0])/popt_exponential[1]
xhalf = -np.log(2)/popt_exponential[1]

AttributeError: 'DataFrame' object has no attribute 'probs'

In [13]:
np.isnan(np.nan)

True

In [14]:
r = np.arange(0., data.radius.max(), 1.)
p = exponential(r, popt_exponential[0], popt_exponential[1], popt_exponential[2])

#%matplotlib widget
fig = plt.figure(figsize=(4,2.5))
ax1 = fig.add_subplot(111)
data.plot.scatter(x='radius', y='probs', logy=False, ax=ax1)
ax1.plot(r, p, 'r')
#ax1.plot(r, (p-popt_exponential[2])/326.59, 'r')
ax1.vlines(xhalf, ymin=0., ymax=y0/2, color='k', ls='--')
ax1.hlines(y0/2, xmin=0., xmax=xhalf, color='k', ls='--')
ax1.set_title('Probability decayed by half')
#fig.savefig('images_res/half_decay_'+str(src_ID)+'_exp.png', bbox_inches = 'tight', dpi=200)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: 'probs'

In [15]:
probs_tw[src_ID] = 1.
ratios_bool = np.reshape(probs_tw,(ydim,xdim))

fig = plt.figure(figsize=(10,10))
ax_im = fig.add_subplot(111)

print(np.max(np.max(ratios_bool)))
im = ax_im.imshow(np.flipud(ratios_bool)/np.max(np.max(ratios_bool)), 
    vmin=0., vmax=1.,  
    extent=[xmin, xmax, ymin, ymax], 
    interpolation='nearest',
    cmap='BuPu')
c0 = ax_im.contour(xx, yy, np.flipud(topo),levels, colors='C5', alpha=0.5,linewidths=0.5)

stas_array = ['BON','BOR','DSO','SNE']

bbox = dict(boxstyle='round', fc='1',alpha=0.7,lw=0)
ax_im.scatter(srcs[:,0], srcs[:,1],
        marker='^', c='C2', s=120, edgecolor='k',linewidth=0.3)
for iista, sta in enumerate(stas_array):
    if iista == 0:
        xanno = -300.
        yanno =   20.
    elif iista == 1:
        xanno = -140.
        yanno = -150.
    elif iista == 2:
        xanno =   70.
        yanno = -110.
    elif iista == 3:
        xanno = -300.
        yanno =  -30.
    anno = (stas_array[iista])
    ax_im.annotate(anno, xy=(srcs[iista,0],srcs[iista,1]),
            xytext=(srcs[iista,0]+xanno,srcs[iista,1]+yanno),
            size=12, color='C2')

msize = 120. 
mlw   = 2.
ax_im.scatter(pickX,pickY, 
        facecolors='none', edgecolors='C3', 
        s = msize, linewidth=mlw)

ax_im.set_xlim(xx[0]+400., xx[-1]-200.)
ax_im.set_ylim(yy[0]+300., yy[-1]-250.)


#INSET
zoom = 2.5
axins = zoomed_inset_axes(ax_im, zoom, loc=5)
axins.imshow(np.flipud(ratios_bool)/np.max(np.max(ratios_bool)), #20., 
             vmin=0., vmax=1.,
             extent=[xmin, xmax, ymin, ymax],
             interpolation='nearest',
             cmap='BuPu', 
             aspect='equal')
axins.contour(xx, yy, np.flipud(topo),levels, colors='C5', alpha=0.5,linewidths=0.5)
axins.scatter(pickX,pickY,
              facecolors='none', edgecolors='C3',
              s = 2*msize*zoom, linewidth=mlw)

# sub region of the original image
x1, x2, y1, y2 = pickX-100., pickX+100., pickY-100., pickY+100.
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticks([])
axins.set_yticks([])
mark_inset(ax_im, axins, loc1=2, loc2=3, fc="none", ec="0.5")


ax_cb = fig.add_axes([0.91,0.195,0.02,0.6])
cb = plt.colorbar(im, cax=ax_cb, ticks=[ 0, 1])
ax_cb.set_title('PDF')

ax_im.set_xticks([])
ax_im.set_yticks([])
fig.subplots_adjust(wspace=0.03)
plt.show()
#fig.savefig('landscape_'+freqID+'_'+chID+'_'+simu_case[1:5]+'.pdf', bbox_inches = 'tight')

NameError: name 'probs_tw' is not defined